In [1]:
import os
os.sys.path.append("C:\\Users\\markp\\repos\\PlittGiocomo_CA1Morph_2019")
import numpy as np
import scipy as sp
from scipy import signal
import sklearn as sk
from sklearn import neighbors
import preprocessing as pp
import utilities as u
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pickle
from scipy.interpolate import interp1d as spline
from sklearn.linear_model import HuberRegressor as hreg
from sklearn.linear_model import LinearRegression as lreg
from sklearn.decomposition import PCA as pca
from scipy.optimize import minimize
import umap
import umap.plot

%matplotlib inline

%load_ext autoreload
%autoreload 2

C:\Users\markp\anaconda3\lib\site-packages\xarray\core\merge.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [2]:
df = pp.load_session_db()
df = df[df['RewardCount']>20]
df = df.sort_values(['MouseName','DateTime','SessionNumber'])
df = df[df["Track"]=="TwoTower_foraging"]
df = df[df["Imaging"]==1]

In [3]:
df_mouse = df[df["MouseName"]=='4139265.5']
sess = df_mouse.iloc[-2]

In [4]:
with open(os.path.join("D:\\Suite2P_Data\\",sess["MouseName"],"%s_%s_%i.pkl" % (sess["Track"],sess["DateFolder"],sess["SessionNumber"])),'rb') as f:
    data = pickle.load(f)
VRDat,S = data['VRDat'],data['S']
S[np.isnan(S)]=0
S = S/np.percentile(S,95,axis=0)
trial_info,S_trial_mat= data['trial_info'],data['S_trial_mat']
S_trial_mat[np.isnan(S_trial_mat)]=0


In [5]:
print(S_trial_mat.shape)
# S_trial_mat

(120, 45, 932)


In [6]:
S_sm = sp.ndimage.filters.gaussian_filter1d(S,5,axis=0)
S_sm_red = sk.decomposition.PCA(n_components=50).fit_transform(S_sm)

posarray = VRDat['pos']._values
posmask = posarray>0
posarray = posarray[posmask]

In [7]:
mapper =  umap.UMAP(metric='cosine',n_neighbors=50,min_dist=.3,n_components=3).fit(S_sm_red[posmask,:])

In [8]:
%matplotlib qt
# umap.plot.points(mapper)

morpharray = VRDat['morph']._values[posmask]
morpharray[morpharray<0]=0
morpharray[morpharray>1]=1


# posarray[posarray<-50]=-50
f= plt.figure()
# f,ax = plt.subplots()
ax = f.add_subplot(121,projection='3d')
ax.scatter(mapper.embedding_[:,0],mapper.embedding_[:,1],mapper.embedding_[:,2],c=morpharray,cmap='cool')
ax = f.add_subplot(122,projection='3d')
ax.scatter(mapper.embedding_[:,0],mapper.embedding_[:,1],mapper.embedding_[:,2],c=posarray,cmap='magma')
# ax = f.add_subplot(123,projection='3d')
# ax.scatter(mapper.embedding_[:,0],mapper.embedding_[:,1],mapper.embedding_[:,2],c=VRDat['time'],cmap='viridis')

# f,ax = plt.subplots()
# pos_mat = np.linspace(0,450,num=45)[np.newaxis,:]*np.ones(S_trial_mat.shape[:2])
# ax.scatter(mapper.embedding_[:,0],mapper.embedding_[:,1],c=pos_mat.ravel(),cmap='magma')

In [9]:
mapper = umap.UMAP(metric='cosine',n_neighbors=50,min_dist=.3).fit(S_trial_mat.reshape(-1,S_trial_mat.shape[-1]))

In [10]:
%matplotlib qt
# umap.plot.points(mapper)
f,ax = plt.subplots()
morph_mat = trial_info['morphs'][:,np.newaxis]*np.ones(S_trial_mat.shape[:2])

ax.scatter(mapper.embedding_[:,0],mapper.embedding_[:,1],c=morph_mat.ravel(),cmap='cool')

f,ax = plt.subplots()
pos_mat = np.linspace(0,450,num=45)[np.newaxis,:]*np.ones(S_trial_mat.shape[:2])
ax.scatter(mapper.embedding_[:,0],mapper.embedding_[:,1],c=pos_mat.ravel(),cmap='magma')
